In [38]:
import pandas as pd
import matplotlib

In [41]:
%run data.ipynb
%run ../modules/preprocessing.ipynb
%run ../modules/cds.ipynb
%run ../modules/utils.ipynb

In [44]:
# df15 = load_uc2(15)
df15 = pd.read_excel('/Users/bryan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Lys_FA_15.xlsx')
df15 = thermo_df(df15)
df16 = load_uc2(16)
df21 = load_uc2(21)
df22 = load_uc2(22)
df15.shape, df16.shape, df21.shape, df22.shape

((6382, 5), (5217, 5), (4401, 5), (4783, 5))

In [45]:
df15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6382 entries, 0 to 6381
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mass    6382 non-null   float64
 1   RT      6382 non-null   float64
 2   Vol     6382 non-null   float64
 3   RA      6382 non-null   float64
 4   FA      6382 non-null   float64
dtypes: float64(5)
memory usage: 299.2 KB


In [121]:

df_src = df16.copy()
# df_src = df_chosen.copy()
plotly_zone(df_src)

In [124]:
# direct sequencing

df_top = df_src[(df_src.Mass>6000)&(df_src.Mass<8800)].sort_values('Vol', ascending=False).iloc[:100].copy()
# df_top = df_src.copy()
# plotly_zone(df_top4
df_salts = list()
for salt in [21.9819, 37.9558, 34.9694, 27.9949, 56.92]:
    df_salt = match_dfs(df_top, df_top, shift=salt)
#     df_top = df_top.drop(df_salt.index)
    df_salts.append(df_salt)
df_salts = pd.concat(df_salts)
# plotly_zones(df_top, df_salts)

bcr = base_calling_random(df_top)
plotly_basecalling(*bcr, annotate=False)
# groups = basecalling_groups(df_top)
# plotly_basecalling(*groups[0], annotate=False)

dfm1 = match_dfs(bcr[0], df_src, shift=-1*M)
dfm2 = match_dfs(bcr[0], df_src, shift=1*M)
# plotly_multi_zones([bcr[0], dfm1, dfm2])
plotly_zones(bcr[0], dfm1)

mass_x (100,) mass_y (100,)


In [84]:
target = 3982
dfa = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
target += M
dfb = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
dfa, dfb

(             Mass         RT       Vol        RA        FA
 2410  3982.511312  11.223738  33418.40  0.002141  0.000325
 6090  3982.617441   9.627505   2552.98  0.000164  0.000025,
             Mass         RT         Vol        RA        FA
 209  3996.524273  11.437607  3781881.54  0.242237  0.036732)

In [125]:
# Choose dots

idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_top #df_src[(df_src.Mass>5000)&(df_src.Mass<8700)]
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

FigureWidget({
    'data': [{'marker': {'color': [#6371f2, #6371f2, #6371f2, #6371f2, #6371f2,
               …

points [0, 1, 6, 7, 8, 10, 11, 12, 13, 16, 17, 19, 20, 21, 22, 23, 24, 28, 29, 31, 33, 34, 48, 51, 53, 55, 56, 58, 60, 63, 66, 70, 74, 75, 77, 78, 79, 81, 93]


In [126]:
df_chosen = df_sample.iloc[idxs]
# df_zone = df_sample.drop(df_chosen.index)
df_chosen.shape

(39, 5)

In [127]:
df5p = df_chosen.copy()

In [128]:
df_sample = df5p.sort_values('Vol', ascending=False).iloc[:20]
# df_sample = df5p
orientation = 5

# df_sf, df_salts = remove_salts(df_sample)
df_sf = df_sample

bcr = base_calling_random(df_sf)
plotly_basecalling(*bcr, annotate=False, y='RT')
group = bcr

# groups = basecalling_groups(df_sf)
# group = groups[0]
# plotly_basecalling(*group, annotate=False)

# output seats
# df = process_mass_base_seats(*group, full_mass=24174, orientation=orientation)
# df.to_excel('~/Downloads/seats.xlsx')

dfm = match_dfs(bcr[0], df_src, shift=-1*M)
if not dfm.empty:
    plotly_zones(bcr[0], dfm, y='RT')

mass_x (20,) mass_y (20,)


In [131]:
target = 8650
dfa = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
target += M
dfb = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
dfa, dfb

(            Mass         RT       Vol        RA        FA
 2386  8650.19114  15.751516  14874.17  0.002136  0.000266,
              Mass         RT        Vol        RA        FA
 273   8664.202819  15.845092  902358.80  0.129589  0.016159
 3753  8663.067011  16.349804    5876.33  0.000844  0.000105)

In [32]:
# bcr[0].to_excel('~/Downloads/type_position_ori.xlsx')
dfa = bcr[0].copy()
dfb = dfm.copy()
dfa['DeMethyl'] = False
dfb['DeMethyl'] = True
dfab = pd.concat([dfa, dfb])
dfab.to_excel('~/Downloads/type_position_ori.xlsx')

In [34]:
def remove_salts(df_sample, salts=[21.9819, 37.9558, 34.9694, 27.9949, 56.92]):
    df = df_sample.copy()
    df_salts = list()
    for salt in salts:
        df_salt = match_dfs(df, df, shift=salt)
#         df = df.drop(df_salt.index)
        df_salts.append(df_salt)
    df_salts = pd.concat(df_salts).drop_duplicates()
    df = df.drop(df_salts.index)
    return df, df_salts